# Investigation 02 - Training Markov Models

Brian Bahmanyar - Due: Wednesday, April 13

___

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
## Please disregard, just some css for styling
from IPython.display import HTML
HTML("""<style>@import "http://fonts.googleapis.com/css?family=Lato|Source+Code+Pro|Montserrat:400,700";#notebook-container{-webkit-box-shadow:none;box-shadow:none}h1,h2,h3,h4,h5,h6{font-family:'Avenir Next'}h1{font-size:4.5em}h2{font-size:4rem}h3{font-size:3.5rem}h4{font-size:3rem}h5{font-size:2.5rem}h6{font-size:2rem}p{font-family:'Avenir Next';font-size:12pt;line-height:15pt;color:#2F4F4F}.CodeMirror pre{font-family:'Source Code Pro', monospace;font-size:0.95em}div.input_area{border:none;background:whitesmoke}</style>""")

___

# The Data

All the markov models will be trained from the coprus in [trump.txt](./trump.txt) which comprises two of Donald Trump's speeches hosted by the Washington Post ([Article 1](https://www.washingtonpost.com/news/post-politics/wp/2015/06/16/full-text-donald-trump-announces-a-presidential-bid/), [Article 2](https://www.washingtonpost.com/news/post-politics/wp/2016/02/20/transcript-donald-trumps-victory-speech-after-the-south-carolina-gop-primary/)). I scraped, cleaned, and tokenized the text from the web pages' HTML, then wrote it to [trump.txt](./trump.txt).

In [3]:
%%bash
wc -lw trump.txt

     167    8264 trump.txt


The corpus contains 8264 words in total, devided among 167 lines.

___

# The Markov Models

In [4]:
## I'm importing functions from the python scripts for use here without copying and pasting
##     all the functions. Please refer to the .py files for the implementations.
from train_markov_chain import get_transition_matrix
from generate_text import simulate_markov_states, get_text

**Zero-th Order Markov Model (Uni-Gram Model)**

He we will not build a markov chain at all. To create a benchmark for our Markov Models to come, lets see what randomly picking 200 words from our file looks like as a paragraph of text.

In [5]:
%%bash --out paragraph
for i in `seq 1 20000`;
    do
        line_string=$(head -$((${RANDOM} % `wc -l < trump.txt` + 1)) trump.txt | tail -1)
        line_array=($line_string)
        num_words=${#line_array[*]}
        echo ${line_array[$((RANDOM%num_words))]}
    done

In [6]:
' '.join(paragraph.split('\n')[:200])

'fast have people the to what billion china me come from be going you right believe zero donnie in ford a where wrong fast the oh interests really a third of will say oil wont hampshire before trader so so you on they a they just it youre cannot charge cut also love leaders choice that bad thats getting banks the get ships trump have trump total 20000 and by were do is to that his to congratulate web never world people person real needs you costs and to iran york it there jobs cant think gun another him a to some by for said to iraq tells up to you and leader wounded in tower it in so are to will our do with so really we a when mr all send its to and bridges great of patients build we work 35 love stupidity we have head making now nevada from so getting rates how by military me so are building they very iran better youre before good candidates tax working need understand happened and leadership life have be husband thousands there we to us they broadcast are you of these careful one it

To get a sense of the estmated word frquency from text generated by arbitrarily picking words from the corpus, lets look at the 10 most frequently occuring words over the 20000 simulated words.

In [7]:
word_count = Counter(paragraph.split('\n'))

In [8]:
list(zip(range(1,11), word_count.most_common(10)))

[(1, ('the', 773)),
 (2, ('and', 665)),
 (3, ('to', 631)),
 (4, ('a', 475)),
 (5, ('we', 459)),
 (6, ('have', 341)),
 (7, ('of', 332)),
 (8, ('it', 323)),
 (9, ('you', 322)),
 (10, ('they', 316))]

Typical words common in the english language (and therefore most frequent in our corpus) apprear the most. The top 3 are 'the', 'and', and 'to' respectively.

**First Order Markov Model (Bi-Gram Model)**

In [9]:
P = get_transition_matrix('trump.txt', markov_model_order=1)

In [10]:
P.shape

(1265, 1265)

Our transition probability matrix is square, as expected, and our state space is 1265 states large.

Obviously this transition matrix is going to be very sparse, so showing a slice of it won't be very helpful. Instead I'll display a subset of the state space and confirm that the first 10 row probabilities sum to one:

In [11]:
P.index

Index(['('1',)', '('10',)', '('100',)', '('10000',)', '('12',)', '('1290',)',
       '('13',)', '('15',)', '('16',)', '('18',)',
       ...
       '('yesterday',)', '('yet',)', '('york',)', '('you',)', '('young',)',
       '('your',)', '('youre',)', '('yourself',)', '('youve',)', '('zero',)'],
      dtype='object', length=1265)

In [12]:
P.sum(axis=1)[:10]

('1',)        1
('10',)       1
('100',)      1
('10000',)    1
('12',)       1
('1290',)     1
('13',)       1
('15',)       1
('16',)       1
('18',)       1
dtype: float64

Now that we have confirmed that our transition matrix is looking good, we can start using it to generate text that resembles the structure of Donald Trump's speeches.

In [13]:
print(' '.join(get_text(simulate_markov_states(P, num_states=200))))

wanted so bloated be going to insist on education that costs me back to bring it great deals even waste and stronger than us economically they will not even better and then they give you know that once they always losing this made a few weeks because if run and with everything everything will do well if a statistic thats the wall and wouldnt let me it recently and better and therefore make this and we really good so ladies and ill say no thats going to negotiate with this man tells me so because we expect to come in in our country could we need money were left behind for public office just announced our theme you its a great again because im not be there very close more than us economically the officials actually a wonderful things for our leaders that can take care of the price up because boy does that had so many web site 5 billion but look at rubio he didnt know were becoming a country needs a web site have a little boost last moment this is that are these people and were going to b

Using a first order Markov Model, and generating 200 states, the text generated is pretty scatted and hard to follow (although in the models defense so is the corpus itself).

To get a sense of the word frequencies this model is outputing were going to need to simulate much more text. Below I will simulate 100,000 states and display the 10 most frequent words, and their counts.

In [14]:
word_count.clear()
word_count.update(get_text(simulate_markov_states(P, num_states=100000)))

In [15]:
list(zip(range(1,11), word_count.most_common(10)))

[(1, ('the', 3708)),
 (2, ('and', 3505)),
 (3, ('to', 2910)),
 (4, ('a', 2284)),
 (5, ('we', 2231)),
 (6, ('of', 1703)),
 (7, ('you', 1654)),
 (8, ('have', 1602)),
 (9, ('they', 1572)),
 (10, ('it', 1541))]

The frequency of simulated words from this model is the same as our O Order Markov "Model" above. This frequency will not change from model to model as it reflects the marginal word distribution in the corpus itself.

**Second Order Markov Model (Tri-Gram Model)**

In [16]:
P = get_transition_matrix('trump.txt', markov_model_order=2)

In [17]:
P.shape

(5310, 5310)

Our transition probability matrix is square, as expected, and our state space is 5310 states large. It makes sense that we have a larger state space here as were comparing pairs of words.

Below is a subset of the state space:

In [18]:
P.index

Index(['('1', 'billion')', '('10', 'billion')', '('10', 'feet')',
       '('10', 'to')', '('100', 'percent')', '('10000', 'we')',
       '('12', 'billion')', '('1290', 'avenue')', '('13', 'trillion')',
       '('15', 'million')',
       ...
       '('youre', 'not')', '('youre', 'right')', '('yourself', 'how')',
       '('youve', 'seen')', '('zero', 'chance')', '('zero', 'horrible')',
       '('zero', 'ill')', '('zero', 'our')', '('zero', 'tax')',
       '('zero', 'whoever')'],
      dtype='object', length=5310)

Lets generate 200 states using our Second Order Model:

In [19]:
print(' '.join(get_text(simulate_markov_states(P, num_states=200))))

all remember new hampshire we cant they cant get jobs because there are ships at sea and this was during the worst since 1978 but think the most and were going to start we are going to be so much less expensive for people up 29 39 49 and even modestly successful cannot run for public office just cant happen and people are incredible and our real unemployment is anywhere from 18 to 19 and will tell you this story friend of mine is a room loves the second amendment now its very easy the other people said hell never run and one of them are like third world country and you say you cant make a billion dollars a day whenever they have now theyre going to start we are going for people up 29 39 49 and even modestly successful cannot run for public office just cant do that to ford because ford takes care of our vets have been paying it for years and those people are saying whats going to be doing that and promise will never ever forget south carolina were going to know unless run because im tot

This text generated from our Second Order Markov Model here is much more readable and closer to real language than the First Order Model. This is expected because we are breaking fewer links between the words from the original corpus.

To get a sense of the word frequencies this model is outputing were going to need to simulate much more text. Below I will simulate 100,000 states and display the 10 most frequent words, and their counts.

In [20]:
word_count.clear()
word_count.update(get_text(simulate_markov_states(P, num_states=100000)))

In [21]:
list(zip(range(1,11), word_count.most_common(10)))

[(1, ('the', 3775)),
 (2, ('and', 3345)),
 (3, ('to', 2748)),
 (4, ('we', 2336)),
 (5, ('a', 2237)),
 (6, ('of', 1777)),
 (7, ('you', 1659)),
 (8, ('have', 1579)),
 (9, ('they', 1516)),
 (10, ('that', 1509))]

As expected, again, the most frequent words generated are 'the', 'and', 'to'. This is a simulation of the non-conditional frequency of the words from the corpus, so with enough state simulations this should not change from model to model.

**Third Order Markov Model (4-Gram Model)**

In [22]:
P = get_transition_matrix('trump.txt', markov_model_order=3)

In [23]:
P.shape

(7272, 7272)

In [24]:
print(' '.join(get_text(simulate_markov_states(P, num_states=200))))

money if the right person asked them theyd pay a fortune they wouldnt be there except for us and believe me you look at these airports we are like a third world country you look at the increases in your obamacare twentyfive 35 45 even 55 percent and deductibles are through the roof you have to announce and certify to all sorts of wonderful things will happen and people are saying whats going on just want a job and thats whats happening and its got to stop fast islamic terrorism is eating up large portions of the middle east but we dont know how to do it they dont want bureaucrats in washington telling them how to educate their children we spend as a nation more for education per pupil than any other nation in the world per capita than any nation in the world and put them one for each country believe me folks we will do very very well very very well think were going to buy were going to do very very well think were going to go to a plan thats going to be an election in my opinion the ne

This is the most coherent simulated text, which is expected. As the order of the markov model increases we break fewer links between the words from the corpus. This means the generated text will be more natural and readable, but will become increasingly similar to the original corpus.